<a href="https://colab.research.google.com/github/matthewshawnkehoe/Machine-Learning-in-Python/blob/main/chapter13_best-practices-for-the-real-world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Best practices for the real world

You've come far since the beginning of this book. You can now train image classification models, image segmentation models, models for classification or regression on vector data, timeseries forecasting models, text-classification models, sequence-to-sequence models, and even generative models for text and images. You've got all
the bases covered.

However,  your  models  so  far  have  all  been  trained  at  a  small  scale—on  small datasets,  with  a  single  GPU—and  they  generally  haven't  reached  the  best  achievable performance on each dataset we looked at. This book is, after all, an introductory book. If you are to go out in the real world and achieve state-of-the-art results on brand new problems, there's still a bit of a chasm that you'll need to cross.
  
This  penultimate  chapter  is  about  bridging  that  gap  and  giving  you  the  best practices  you'll  need  as  you  go  from  machine  learning  student  to  fully  fledged machine learning engineer. We'll review essential techniques for systematically improving  model  performance:  hyperparameter  tuning  and  model  ensembling.  Then we'll
look at how you can speed up and scale up model training, with multi-GPU and TPU training, mixed precision, and leveraging remote computing resources in the cloud.

## Getting the most out of your models

Blindly  trying  out  different  architecture  configurations  works  well  enough  if  you just need something that works okay. In this section, we'll go beyond "works okay" to "works  great  and  wins  machine  learning  competitions"  via  a  set  of  must-know techniques for building state-of-the-art deep learning models.

### Hyperparameter optimization

When  building  a  deep  learning  model,  you  have  to  make  many  seemingly  arbitrary decisions: How many layers should you stack? How many units or filters should go in each layer? Should you use `relu` as activation, or a different function? Should you use `BatchNormalization` after a given layer? How much dropout should you use? And so on. These architecture-level parameters are called `hyperparameters` to distinguish them from the `parameters` of a model, which are trained via backpropagation.

In practice, experienced machine learning engineers and researchers build intuition  over  time  as  to  what  works  and  what  doesn't  when  it  comes  to these  choices— they develop hyperparameter-tuning skills. But there are no formal rules. If you want to get to the very limit of what can be achieved on a given task, you can't be content
with such arbitrary choices. Your initial decisions are almost always suboptimal, even if you have very good intuition. You can refine your choices by tweaking them by hand and  retraining  the  model  repeatedly—that's  what  machine  learning  engineers  and researchers spend most of their time doing. But it shouldn't be your job as a human to fiddle with hyperparameters all day—that is better left to a machine.

Thus you need to explore the space of possible decisions automatically, systematically, in a principled way. You need to search the architecture space and find the best-performing architectures empirically. That's what the field of automatic hyperparameter optimization is about: it's an entire field of research, and an important one.

The process of optimizing hyperparameters typically looks like this:

1. Choose a set of hyperparameters (automatically).
2. Build the corresponding model.
3. Fit it to your training data, and measure performance on the validation data.
4. Choose the next set of hyperparameters to try (automatically).
5. Repeat.
6. Eventually, measure performance on your test data.

The key to this process is the algorithm that analyzes the relationship between validation  performance  and  various  hyperparameter  values  to  choose  the  next  set  of hyperparameters to evaluate. Many different techniques are possible: Bayesian optimization, genetic algorithms, simple random search, and so on.

Training the weights of a model is relatively easy: you compute a loss function on a mini-batch  of  data  and  then  use  backpropagation  to  move  the  weights  in  the  right direction. Updating hyperparameters, on the other hand, presents unique challenges.

Consider these points:

* The  hyperparameter  space  is  typically  made  up  of  discrete  decisions  and thus isn't continuous or differentiable. Hence, you typically can't do gradient descent in hyperparameter space. Instead, you must rely on gradient-free optimization techniques, which naturally are far less efficient than gradient descent.
* Computing  the  feedback  signal  of  this  optimization  process  (does  this  set  of hyperparameters lead to a high-performing model on this task?) can be extremely expensive: it requires creating and training a new model from scratch on your dataset.
* The feedback signal may be noisy: if a training run performs 0.2% better, is that because of a better model configuration, or because you got lucky with the initial weight values?

Thankfully, there's  a  tool  that  makes  hyperparameter  tuning  simpler:  KerasTuner. Let's check it out.

#### Using KerasTuner

Let's start by installing KerasTuner:

In [1]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.0 MB/s eta 0:00:00


KerasTuner  lets  you  replace  hard-coded  hyperparameter  values,  such  as  `units=32`, with a range of possible choices, such as `Int(name="units", min_value=16,max_value=64, step=16)`. This set of choices in a given model is called the search space of the hyperparameter tuning process.

To specify a search space, define a model-building function (see the listing below). It  takes  an  hp  argument,  from  which  you  can  sample  hyperparameter  ranges,  and  it returns a compiled Keras model.

**A KerasTuner model-building function**

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)           # Sample hyperparameter values from the hp object. After sampling, these values
    model = keras.Sequential([                                                  # (such as the "units" variable here) are just regular Python constants.
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])         # Different kinds of hyperparameters are available: Int, Float, Boolean, Choice.
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model                                                                # The function returns a compiled model.

If you want to adopt a more modular and configurable approach to model-building,
you can also subclass the `HyperModel` class and define a `build` method, as follows.

**A KerasTuner `HyperModel`**

In [3]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):                                            # Thanks to the object-oriented approach, we can configure model constants as
        self.num_classes = num_classes                                          # constructor arguments (instead of hardcoding them in the model-building function).

    def build(self, hp):                                                        # The build() method is identical to our prior  build_model() standalone function.
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

<ipython-input-3-643dcc05e516>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


The next step is to define a "tuner." Schematically, you can think of a tuner as a for loop that will repeatedly

* Pick a set of hyperparameter values
* Call the model-building function with these values to create a model
* Train the model and record its metrics

KerasTuner has several built-in tuners available—`RandomSearch`, `BayesianOptimization`, and `Hyperband`. Let's try `BayesianOptimization`, a tuner that attempts to make smart  predictions  for  which  new hyperparameter  values  are  likely  to  perform  best given the outcomes of previous choices:

In [4]:
tuner = kt.BayesianOptimization(
    build_model,                                                                # Specify the model-building function (or hyper-model instance).
    objective="val_accuracy",                                                   # Specify the metric that the tuner will seek to optimize. Always specify validation metrics,
                                                                                # since the goal of the search process is to find models that generalize!
    max_trials=100,                                                             # Maximum number of different model configurations (“trials”) to try before ending the search.
    executions_per_trial=2,                                                     # To reduce metrics variance, you can train the same model multiple times and average the results.
                                                                                # executions_per_trial is how many training rounds (executions) to run for each model configuration (trial).
    directory="mnist_kt_test",                                                  # Where to store the search logs.
    overwrite=True,                                                             # Whether to overwrite data in directory to start a new search. Set this to True if you’ve modified
                                                                                # the model-building function, or to False to resume a previously started search with the same
                                                                                # model-building function.
)

You can display an overview of the search space via `search_space_summary()`:

In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


### Objective maximization and minimization
For built-in metrics (like accuracy, in our case), the *direction* of the metric (accuracy should  be  maximized,  but  a  loss  should  be  minimized)  is  inferred  by  KerasTuner. However, for a custom metric, you should specify it yourself, like this:

In [ ]:
# Pseudocode
objective = kt.Objective(
    name="val_accuracy",                                                        # The metric’s name, as found in epoch logs
    direction="max")                                                            # The metric’s desired direction: "min" or "max"
tuner = kt.BayesianOptimization(
    build_model,
    objective=objective,
    ...
)

Finally, let's launch the search. Don't forget to pass validation data, and make sure not to use your test set as validation data—otherwise you'd quickly start overfitting to your test data, and you wouldn't be able to trust your test metrics anymore:

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]                                                       # Reserve for later
y_train_full = y_train[:]                                                       # Reserve for later
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]         # Set aside as a validation set
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]         # Set aside as a validation set
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),              # Use an EarlyStopping callback to stop training when you start overfitting
]
tuner.search(                                                                   # This takes the same arguments as fit() (it simply passes them down to fit() for each new model).
    x_train, y_train,
    batch_size=128,
    epochs=100,                                                                 # Use a large number of epochs (you don’t know in advance how many epochs each model will need)
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 46 Complete [00h 01m 17s]
val_accuracy: 0.9743500053882599

Best val_accuracy So Far: 0.9763500094413757
Total elapsed time: 00h 57m 18s

Search: Running Trial #47

Value             |Best Value So Far |Hyperparameter
64                |64                |units
adam              |adam              |optimizer

Epoch 1/100
391/391 - 3s - loss: 0.4638 - accuracy: 0.8765 - val_loss: 0.2473 - val_accuracy: 0.9297 - 3s/epoch - 7ms/step
Epoch 2/100
391/391 - 1s - loss: 0.2194 - accuracy: 0.9374 - val_loss: 0.1785 - val_accuracy: 0.9499 - 1s/epoch - 3ms/step
Epoch 3/100
391/391 - 1s - loss: 0.1666 - accuracy: 0.9523 - val_loss: 0.1473 - val_accuracy: 0.9588 - 1s/epoch - 4ms/step
Epoch 4/100
391/391 - 1s - loss: 0.1353 - accuracy: 0.9612 - val_loss: 0.1343 - val_accuracy: 0.9613 - 1s/epoch - 3ms/step
Epoch 5/100
391/391 - 1s - loss: 0.1146 - accuracy: 0.9668 - val_loss: 0.1241 - val_accuracy: 0.9634 - 1s/epoch - 3ms/step
Epoch 6/100
391/391 - 2s - loss: 0.0993 - accuracy: 0.9717 - val_los

The  preceding  example  will  run  in  just  a  few  minutes,  since  we're  only  looking  at  a few  possible  choices  and  we're  training  on  MNIST.  However,  with  a  typical  search space  and  dataset,  you'll  often  find  yourself  letting  the  hyperparameter  search  run overnight  or  even  over  several  days.  If  your  search  process  crashes,  you  can  always restart  it—just  specify `overwrite=False`  in  the  tuner  so  that  it  can  resume  from the trial logs stored on disk.

Once  the  search  is  complete,  you  can  query  the  best  hyperparameter  configurations, which you can use to create high-performing models that you can then retrain.

**Querying the best hyperparameter configurations**

In [ ]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)                                # Returns a list of HyperParameter objects, which you can pass to the model-building function

Usually, when retraining these models, you may want to include the validation data as part  of  the  training  data,  since  you  won't  be  making  any  further  hyperparameter changes,  and  thus  you  will  no  longer  be   evaluating  performance  on  the  validation data.  In  our  example, we'd  train  these  final  models  on  the  totality  of  the  original
MNIST training data, without reserving a validation set.

Before we can train on the full training data, though, there's one last parameter we need to settle: the optimal number of epochs to train for. Typically, you'll want to train the  new  models  for  longer  than  you  did  during  the  search:  using  an  aggressive `patience` value in the `EarlyStopping` callback saves time during the search, but it may lead to under-fit models. Just use the validation set to find the best epoch:

In [ ]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)                        # Note the high patience value
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

Finally,  train  on  the  full  dataset  for  just  a  bit  longer  than  this  epoch  count, since you're training on more data; 20% more in this case:

In [ ]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Note that if you're not worried about slightly underperforming, there's a shortcut you can take: just use the tuner to reload the top-performing models with the best weights saved during the hyperparameter search, without retraining new models from scratch:

In [ ]:
best_models = tuner.get_best_models(top_n)

**Remark:** One  important  issue  to  think  about  when  doing  automatic  hyperparameter  optimization  at  scale  is  validation-set  overfitting.  Because  you're updating hyperparameters based on a signal that is computed using your validation data, you're effectively training them on the validation data, and thus they will quickly overfit to the validation data. Always keep this in mind.

#### The art of crafting the right search space

Overall,  hyperparameter  optimization  is  a  powerful  technique  that  is  an  absolute requirement for getting to state-of-the-art models on any task or to win machine learning competitions. Think about it: once upon a time, people handcrafted the features that  went  into  shallow  machine  learning  models.  That  was  very  much  suboptimal. Now, deep learning automates the task of hierarchical feature engineering—features are learned using a feedback signal, not hand-tuned, and that's the way it should be. In the same way, you shouldn't handcraft your model architectures; you should optimize them in a principled way.

However, doing hyperparameter tuning is not a replacement for being familiar
with model architecture best practices. Search spaces grow combinatorially with the number of choices, so it would be far too expensive to turn everything into a hyperparameter  and  let  the  tuner  sort  it  out.  You  need  to  be  smart  about  designing  the right  search  space. Hyperparameter  tuning is  automation,  not  magic:  you  use it to automate  experiments  that  you  would  otherwise  have  run  by  hand,  but  you  still need  to  handpick  experiment  configurations  that  have  the  potential  to  yield  good
metrics.

The  good  news  is  that  by  leveraging  hyperparameter  tuning,  the  configuration decisions you have to make graduate from micro-decisions (what number of units do I pick for this layer?) to higher-level architecture decisions (should I use residual connections throughout this model?). And while micro-decisions are specific to a certain model and a certain dataset, higher-level decisions generalize better across different tasks  and  datasets.  For  instance,  pretty  much  every  image  classification  problem  can be solved via the same sort of search-space template.

Following this logic, KerasTuner attempts to provide premade search spaces that are relevant to broad categories of problems, such as image classification. Just add data, run the search, and get a pretty good model. You can try the hypermodels `kt.applications.HyperXception` and `kt.applications.HyperResNet`,  which are effectively tunable versions of Keras Applications models.

#### The future of hyperparameter tuning: automated machine learning

Currently, most of your job as a deep learning engineer consists of munging data with Python scripts and then tuning the architecture and hyperparameters of a deep network at length to get a working model, or even to get a state-of-the-art model, if you are  that  ambitious.  Needless  to  say,  that  isn't  an  optimal  setup.  But  automation  can help, and it won't stop merely at hyperparameter tuning.

Searching over a set of possible learning rates or possible layer sizes is just the first step. We can also be far more ambitious and attempt to generate the model architecture itself from scratch, with as few constraints as possible, such as via reinforcement learning or genetic algorithms. In the future, entire end-to-end machine learning pipelines will be automatically generated, rather than be handcrafted by engineer-artisans. This is called automated machine learning, or AutoML. You can already leverage libraries like  AutoKeras  (https://github.com/keras-team/autokeras) to solve basic machine learning problems with very little involvement on your part.

Today, AutoML  is  still in its  early  days,  and  it  doesn't  scale to  large  problems.  But when AutoML becomes mature enough for widespread adoption, the jobs of machine learning engineers won't disappear—rather, engineers will move up the value-creation chain. They will begin to put much more effort into data curation, crafting complex loss functions  that  truly  reflect  business  goals,  as  well  as  understanding  how  their  models
impact the digital ecosystems in which they're deployed (such as the users who consume the model's predictions and generate the model's training data). These are problems that only the largest companies can afford to consider at present.

Always look at the big picture, focus on understanding the fundamentals, and keep in mind that the highly specialized tedium will eventually be automated away. See it as a gift—greater productivity for your workflows—and not as a threat to your own relevance. It shouldn't be your job to tune knobs endlessly.

### Model ensembling

Another powerful technique for obtaining the best possible results on a task is *model ensembling*. Ensembling consists of pooling together the predictions of a set of different  models  to  produce  better  predictions.  If  you  look  at  machine  learning  competitions, in particular on Kaggle, you'll see that the winners use very large ensembles of models that inevitably beat any single model, no matter how good.

Ensembling  relies  on  the  assumption  that  different  well-performing  models trained independently are likely to be good for *different reasons*: each model looks at slightly different aspects of the data to make its predictions, getting part of the "truth" but not all of it. You may be familiar with the ancient parable of the blind men and the elephant: a group of blind men come across an elephant for the first time and try to understand what the elephant is by touching it. Each man touches a different part of the elephant's body—just one part, such as the trunk or a leg. Then the men describe
to each other what an elephant is: "It's like a snake," "Like a pillar or a tree," and so on. The blind men are essentially machine learning models trying to understand the manifold  of  the  training  data,  each  from  their  own  perspective,  using  their  own assumptions (provided by the unique architecture of the model and the unique random weight initialization). Each of them gets part of the truth of the data, but not the whole truth. By pooling their perspectives together, you can get a far more accurate description of the data. The elephant is a combination of parts: not any single blind man gets it quite right, but, interviewed together, they can tell a fairly accurate story.

Let's use classification as an example. The easiest way to pool the predictions of a set of classifiers (to *ensemble the classifiers*) is to average their predictions at inference time:

In [ ]:
# Psuedocode
preds_a = model_a.predict(x_val)                                                # Use four different models to compute initial predictions.
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)
final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)                    # This new prediction array should be more accurate than any of the initial ones.

However, this will work only if the classifiers are more or less equally good. If one of them is significantly worse than the others, the final predictions may not be as good as the best classifier of the group.

A  smarter  way  to  ensemble  classifiers  is  to  do  a  weighted  average,  where  the weights are learned on the validation data—typically, the better classifiers are given a higher weight, and the worse classifiers are given a lower weight. To search for a good set of ensembling weights, you can use random search or a simple optimization algorithm, such as the Nelder-Mead algorithm:

In [ ]:
# Psuedocode
preds_a = model_a.predict(x_val)                                                # Use four different models to compute initial predictions.
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)
final_preds = 0.5 * preds_a + 0.25 * preds_b + 0.1 * preds_c + 0.15 * preds_d   # These weights (0.5, 0.25, 0.1, 0.15) are assumed to be learned empirically.

There are many possible variants: you can do an average of an exponential of the predictions,  for  instance.  In  general,  a  simple  weighted  average  with  weights  optimized on the validation data provides a very strong baseline.

The key to making ensembling work is the *diversity* of the set of classifiers. Diversity is strength. If all the blind men only touched the elephant's trunk, they would agree that elephants are like snakes, and they would forever stay ignorant of the truth of the elephant. Diversity is what makes ensembling work. In machine learning terms, if all of your models are biased in the same way, your ensemble will retain this same bias. If your models are *biased in different ways*, the biases will cancel each other out, and the ensemble will be more robust and more accurate.

For this reason, you should ensemble models that are *as good as possible* while being *as different as possible*. This typically means using very different architectures or even different brands of machine learning approaches. One thing that is largely not worth doing is ensembling the same network trained several times independently, from different random initializations. If the only difference between your models is their random initialization and the order in which they were exposed to the training data, then your ensemble will be low-diversity and will provide only a tiny improvement over any single model.

One  thing  I  have  found  to  work  well  in  practice—but  that  doesn't  generalize  to every problem domain—is using an ensemble of tree-based methods (such as random forests  or  gradient-boosted  trees)  and  deep  neural  networks.  In  2014,  Andrey  Kolev and  I  took  fourth  place  in  the  Higgs  Boson  decay  detection  challenge  on  Kaggle (www.kaggle.com/c/higgs-boson) using an ensemble of various tree models and deep neural  networks.  Remarkably,  one  of the  models  in the  ensemble  originated  from  a different method than the others (it was a regularized greedy forest), and it had a significantly worse score than the others. Unsurprisingly, it was assigned a small weight in the ensemble. But to our surprise, it turned out to improve the overall ensemble by a large factor, because it was so different from every other model: it provided information that the other models didn't have access to. That's precisely the point of ensembling. It's not so much about how good your best model is; it's about the diversity of your set of candidate models.

## Scaling-up model training

### Speeding up training on GPU with mixed precision

Recall the "loop of progress" concept we introduced in chapter 7: the quality of your ideas is a function of how many refinement cycles they've been through (see figure below). And the speed at which you can iterate on an idea is a function of how fast you can set up an experiment, how fast you can run that experiment, and finally, how well you can analyze the resulting data.

#### Understanding floating-point precision

In [ ]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

In [ ]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

#### Mixed-precision training in practice

In [ ]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

### Multi-GPU training

#### Getting your hands on two or more GPUs

#### Single-host, multi-device synchronous training

### TPU training

#### Using a TPU via Google Colab

#### Leveraging step fusing to improve TPU utilization

## Summary